In [1]:
import pandas as pd
import numpy as np
from maxijdefs import *

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

In [2]:
def logbin(f,p,df):
    #df = step size in log frequency
    fx=np.zeros(len(f))
    px=np.zeros(len(p))
    fx[:]=f[:]
    fx[0]=fx[1]
    p0=p[0]
    px[:]=p[:]
    px[0]=0.

    fl=np.log10(fx)
    nel=len(f)

    fmin=fl[1]
    fmax=np.max(fl)

    nf=np.round((fmax-fmin)/df).astype(int)
    f2=np.arange(nf)*df+fmin
    p2=np.zeros(nf)

    for i in range(nf):
        fa=10.**f2[i]
        fb=10.**(f2[i]+df)
        fm=(fa+fb)/2.0
        dff=fm-fa
        if len(f[np.abs(f-fm)<=dff])>0:
            p2[i]=np.mean(px[np.abs(f-fm)<=dff])

    return f2,p2

In [3]:
def pds_chunk128(t,flx,t1,t2):
    # takes time series - not lightcurve - time and flx
    # extracts time from t1 to t2
    # calculates power spectrum in 128s segments and returns
    
    t_span=t2-t1
    n_chunk=np.round(t_span/128.0).astype(int)
    
    print t_span, n_chunk
    
    dt2=128.0/2.0 # half-width of chunk
    
    for i in range(n_chunk):
        ty=t-((i*dt2*2)+dt2+t1)
        y=flx[np.abs(ty)<=dt2]
        if i==0:
            flen=len(y)/2
            print i, flen
            p=np.zeros([n_chunk,flen])
            f=np.arange(flen)/(dt2*2)
        af=np.fft.fft(y)
        p[i,:]=np.abs(af[:flen])**2
        
    ptot=np.sum(p,axis=0)
    
    return f,p,ptot


In [4]:
def p_2_logp(f,p,df,n_avg):
    # takes output of pds_chunk128-style program
    # averages over n_avg pieces, and logbins the 2D array
    
    nt=p.shape[0] # number of power spectra
    
    for i in range(nt):
        i1=i-n_avg/2
        i2=i1+n_avg/2
        if i1<0:
            i1=0
        if i2>(nt-1):
            i2=nt-1
        pmean=np.mean(p[i1:i2+1,:],axis=0)/(np.mean(p[i1:i2+1,0])**0.5)
        dumf,dump=logbin(f,pmean,df)
        if i==0:
            nf=len(dump[dump>0])
            p2=np.zeros([nt,nf])
            f2=dumf[dump>0]
        p2[i,:]=dump[dump>0]
        
    return f2,p2

In [6]:
#read data
path = './maxij_data_copies/'
night = '2018-03-28'

#read in data as pandas dataframe:
data = pd.read_pickle(path+'data_'+night+'.pkl')
# print data
fID = data['fileID']

In [7]:
data['maxij_flux'] = data['phot_maxij']
data['tyc_flux'] = data['phot_tyc']
data['ref_flux'] = data['phot_ref3']+data['phot_ref4']+data["phot_ref5"]

In [ ]:
data['clean_flux_bool'] = True
data['localmed_tyc'] = np.nan
data['localmed_maxij'] = np.nan
data['localmed_ref'] = np.nan